In [1]:
import nltk
from nltk.corpus import shakespeare
from nltk.corpus import genesis
from nltk.corpus import gutenberg
from nltk.corpus import brown
from nltk import pos_tag
from nltk import word_tokenize
from nltk import sent_tokenize
from sklearn import datasets, svm, tree, metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
from nltk.book import *
from nltk.tokenize import sent_tokenize, word_tokenize

*** Introductory Examples for the NLTK Book ***
Loading text1, ..., text9 and sent1, ..., sent9
Type the name of the text or sentence to view it.
Type: 'texts()' or 'sents()' to list the materials.
text1: Moby Dick by Herman Melville 1851
text2: Sense and Sensibility by Jane Austen 1811
text3: The Book of Genesis
text4: Inaugural Address Corpus
text5: Chat Corpus
text6: Monty Python and the Holy Grail
text7: Wall Street Journal
text8: Personals Corpus
text9: The Man Who Was Thursday by G . K . Chesterton 1908


### 6.1.1 Consider the following code, it computes 3 different features. Can you tell what each feature computes?

In [2]:
def computeFeature1(text):
    len(text)
    sum = 0;
    for word in text:
        sum+= len(word)
    print(sum)
    return sum/len(text)

Feature1: The ratio of the numbers of characters to the numbers of words in the inputted text.

In [3]:
def getRelevantCategories(text, num):
    taggedWordList = pos_tag(text)
    fd = nltk.FreqDist((t1, t2) for ((w1, t1), (w2, t2)) in nltk.bigrams(taggedWordList))
    res = []
    for (tag, freq) in fd.most_common(num):
        res.append(tag)
    return res

def computeFeature2(text, cats):
    taggedWordList = pos_tag(text)
    fd = nltk.FreqDist((t1, t2) for ((w1, t1), (w2, t2)) in nltk.bigrams(taggedWordList))
    res = []
    for cat in cats:
        res.append(fd.freq(cat))
    return res

# row 6:  res.append(str(tag)) -> res.append(tag)

getRelevantCategories: get num most freqently occuring tag bigrams in the inputted text.

Feature2: The count of each tag bigram(categroy) divided by the total number of inputted tag bigrams(categories), namely the frequency of the each category

In [4]:
def computeFeature3(text):
    taggedWordList = pos_tag(text, tagset='universal')
    fd_tags = nltk.FreqDist(tag for (word, tag) in taggedWordList)
    fd_words = nltk.FreqDist(word for (word, tag) in taggedWordList)
    return fd_words.freq(';')/fd_tags.freq('.')

Feature3: The frequency of ';' in the most frequent words divided by the frequency of '.' in the most frequent tags

In [5]:
df = pd.DataFrame()
num = 300
numTokens = 100
numCategoriesToConsider = 5

# Gold: 300 melville rows + 300 shakespeare rows
df['GOLD'] = ["melville" for i in range(0,num)]+["shakespeare" for i in range(0,num)]

# Add in Feature1
df['FEATURE1'] = [computeFeature1(gutenberg.words('melville-moby_dick.txt')[i*numTokens:(i+1)*numTokens]) for i in range(0,num)]+[computeFeature1(gutenberg.words('shakespeare-hamlet.txt')[i*numTokens:(i+1)*numTokens]) for i in range(0,num)]

# Add in Feature2
allTexts = gutenberg.words('melville-moby_dick.txt')[:num*numTokens]+gutenberg.words('shakespeare-hamlet.txt')[num*numTokens]
tags = getRelevantCategories(allTexts, numCategoriesToConsider)
for f in range(0,numCategoriesToConsider):
    df['FEATURE2_'+str(f)] = [computeFeature2(gutenberg.words('melville-moby_dick.txt')[i*numTokens:(i+1)*numTokens], tags)[f] for i in range(0,num)]+[computeFeature2(gutenberg.words('shakespeare-hamlet.txt')[i*numTokens:(i+1)*numTokens], tags)[f] for i in range(0,num)]

# Add in Feature3
df['FEATURE3'] = [computeFeature3(gutenberg.words('melville-moby_dick.txt')[i*numTokens:(i+1)*numTokens]) for i in range(0,num)]+[computeFeature3(gutenberg.words('shakespeare-hamlet.txt')[i*numTokens:(i+1)*numTokens]) for i in range(0,num)]

# We can inspect what the features look like
print(df[290:310])


# Now we train a classifier similar to what we did for Named Entity classifcation
x = df.iloc[:, 1:len(df.columns)]
y = df.iloc[:, [0]]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

c_tree = DecisionTreeClassifier(max_depth=4)
c_tree.fit(x_train, y_train)

# ... and evaluate it
predicted = list(c_tree.predict(x_test))
gold = list(y_test.loc[:, "GOLD"])
print(classification_report(gold,predicted))

388
358
333
424
386
365
397
382
381
355
361
379
354
373
365
365
367
381
342
341
321
362
374
376
382
385
360
370
387
368
369
309
371
381
421
414
407
295
350
438
360
396
399
399
385
363
365
388
420
399
417
375
376
349
392
397
386
377
331
336
388
388
371
348
389
392
399
405
397
411
451
393
363
386
408
422
373
384
406
362
342
372
352
407
372
404
350
395
392
374
394
435
437
361
416
417
417
394
376
444
394
323
356
364
377
320
351
386
380
408
376
395
374
366
380
337
358
351
368
380
395
338
317
355
327
294
407
349
314
359
356
309
324
394
383
386
365
326
379
352
368
334
369
340
394
363
385
377
373
405
382
397
406
439
374
358
320
326
345
340
306
390
376
374
351
369
415
387
382
387
359
385
395
409
384
381
390
405
410
378
412
356
381
380
368
372
411
390
424
450
472
445
361
384
381
339
408
388
422
405
431
410
373
414
441
439
417
401
384
387
399
394
451
397
396
379
443
403
413
363
360
392
379
371
354
397
346
359
376
381
404
377
373
354
380
371
342
376
365
354
384
425
346
372
383
367
378
423
376
361


Now you extend the classifier with new features. Implement a method computeFeature4 that also takes a list of words as input and returns the average sentence length in tokens as features. 

In [6]:
def computeFeature4(text):
    text_1= ' '.join(text)
    sents = sent_tokenize(text_1)
    count = 0

    for sent in sents:
        sent1 = word_tokenize(sent)
        count += len(sent1)
    return count/len(sents)

In [7]:
# Add in Feature4
df['SENTENCELENGTH'] = [computeFeature4(gutenberg.words('melville-moby_dick.txt')[i*numTokens:(i+1)*numTokens]) for i in range(0,num)]+[computeFeature4(gutenberg.words('shakespeare-hamlet.txt')[i*numTokens:(i+1)*numTokens]) for i in range(0,num)] 

In [8]:
def getMostFrequentPreps(text, num):
    taggedWordList = pos_tag(text, tagset='universal')
    fd = nltk.FreqDist(w for (w, t) in taggedWordList if t == 'ADP')
    preps = []
    for (prep, freq) in fd.most_common(num):
        preps.append(prep)
    return preps

def computeFeature5(text, cats):
    taggedWordList = pos_tag(text, tagset='universal')
    fd = nltk.FreqDist(w for (w, t) in taggedWordList)
    res = []
    for cat in cats:
        res.append(fd.freq(cat))
    return res

In [9]:
#  Add in Feature5
tags = getMostFrequentPreps(allTexts, 5)
print(tags)
for f in range(0,5):
    df['PREP_'+str(f)] = [computeFeature5(gutenberg.words('melville-moby_dick.txt')[i*numTokens:(i+1)*numTokens], tags)[f] for i in range(0,num)]+[computeFeature5(gutenberg.words('shakespeare-hamlet.txt')[i*numTokens:(i+1)*numTokens], tags)[f] for i in range(0,num)]

['of', 'in', 'that', 'with', 'as']


In [10]:
from nltk.corpus import stopwords
stops = stopwords.words('english')

def getMostFrequentContentWords(text, num):
    ConWords = []
    fd = nltk.FreqDist(w for w in text if (w.lower() not in stops) and (w.isalpha()))
    for (Cword, freq) in fd.most_common(num):
        ConWords.append(Cword)
    return ConWords

def computeFeature6(text, cats):
    fd = nltk.FreqDist(w for w in text)
    res = []
    for cat in cats:
        res.append(fd.freq(cat))
    return res

In [11]:
# Add in Feature6
tags = getMostFrequentContentWords(allTexts, 5)
print(tags)
for f in range(0,5):
    df['FUNCTIONWORD_'+str(f)] = [computeFeature6(gutenberg.words('melville-moby_dick.txt')[i*numTokens:(i+1)*numTokens], tags)[f] for i in range(0,num)]+[computeFeature6(gutenberg.words('shakespeare-hamlet.txt')[i*numTokens:(i+1)*numTokens], tags)[f] for i in range(0,num)]

['one', 'like', 'upon', 'sea', 'whale']


In [12]:
# We can inspect what the features look like
print(df[290:310])

            GOLD  FEATURE1  FEATURE2_0  FEATURE2_1  FEATURE2_2  FEATURE2_3  \
290     melville      3.74    0.020202    0.030303    0.010101    0.040404   
291     melville      3.84    0.050505    0.050505    0.010101    0.060606   
292     melville      3.93    0.020202    0.020202    0.020202    0.070707   
293     melville      3.68    0.050505    0.040404    0.010101    0.040404   
294     melville      3.78    0.070707    0.040404    0.040404    0.010101   
295     melville      4.20    0.050505    0.020202    0.030303    0.030303   
296     melville      3.96    0.010101    0.030303    0.000000    0.000000   
297     melville      3.60    0.050505    0.030303    0.030303    0.020202   
298     melville      3.79    0.020202    0.030303    0.010101    0.010101   
299     melville      3.81    0.030303    0.040404    0.040404    0.050505   
300  shakespeare      3.66    0.010101    0.020202    0.000000    0.000000   
301  shakespeare      3.28    0.010101    0.000000    0.000000  

In [13]:
# Now we train a classifier similar to what we did for Named Entity classifcation
x = df.iloc[:, 1:len(df.columns)]
y = df.iloc[:, [0]]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

c_tree = DecisionTreeClassifier(max_depth=4)
c_tree.fit(x_train, y_train)

# ... and evaluate it
predicted = list(c_tree.predict(x_test))
gold = list(y_test.loc[:, "GOLD"])
print(classification_report(gold,predicted))

              precision    recall  f1-score   support

    melville       0.86      0.83      0.85        60
 shakespeare       0.84      0.87      0.85        60

    accuracy                           0.85       120
   macro avg       0.85      0.85      0.85       120
weighted avg       0.85      0.85      0.85       120



Compared with the original one with three given features, the percentage of precision, recall, f1-score is higher after more features are added, which means accuracy of the classifier is higher.